In [1]:
import openpyxl
import pandas as pd
import numpy as np
from numpy.linalg import norm

feature_names = ['Age', 'TB', 'DB', 'Alkphos', 'Sgpt','TP', 'ALB', 'AG_Ratio', 'Gender']
noOfIterations = 100

def convertSign(value):
    if value == "positive":
        return 1
    elif value == "negative":
        return 0
    else:
        return -1

def generateSigns(feature_names):
    dataframe = openpyxl.load_workbook("outputLiverKernelShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_KernelShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_KernelShap = pd.DataFrame(dict)
     
    df_KernelShap.to_excel("signsLiverKernelShap.xlsx")
   
    #################################################################
    dataframe = openpyxl.load_workbook("outputLiverTreeShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_TreeShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3          
          
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_TreeShap = pd.DataFrame(dict)
    
    df_TreeShap.to_excel("signsLiverTreeShap.xlsx")    

    ############################################################################
    dataframe = openpyxl.load_workbook("outputLiverLIMELOOCV_100.xlsx")
    sheet = dataframe.active
    
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_LIME = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 10
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[5].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 5            
            
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_LIME = pd.DataFrame(dict)
   
    df_LIME.to_excel("signsLiverLIME.xlsx")
   

def stackRowsColumns(M, noOfRows, noOfColumns):
    pos = [0] * noOfRows
    for i in range (0, noOfRows):
        pos[i] = i
    
    for i in range(1, noOfRows):
        wsim = [0] * noOfRows
        for j in range(i, noOfRows):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
        aux = pos[max_index]
        pos[max_index] = pos[i]
        pos[i] = aux
    
    for i in range(1, noOfColumns):
        wsim = [0] * noOfColumns
        for j in range(i, noOfColumns):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
    return pos, M

def cosineSimilarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

def similarityCase(M, i):
    s = 0
    k = i - 1
    for j in range (1, k + 1):
        s = s + cosineSimilarity(M.loc[i], M.loc[i - j]) * (1 / j)
    return s

def similarityMatrix(M, noOfRows):
    s = 0
    for i in range(1, noOfRows):
        s = s + similarityCase(M, i)
    s = s / (noOfRows - 1)
    return s

def GlobalAlign(MSP, MS, noOfRows):
    if similarityMatrix(MSP, noOfRows) >  similarityMatrix(MS, noOfRows):
        max = similarityMatrix(MSP, noOfRows)
        min = similarityMatrix(MS, noOfRows)
    else:
        max = similarityMatrix(MS, noOfRows)
        min = similarityMatrix(MSP, noOfRows)
    return min / max
    #return similarityMatrix(MSP, noOfRows) / similarityMatrix(MS, noOfRows)


def createMSP(MP, MS, noOfRows):
    MS_copy = MS.copy(deep = True)
    MP_copy = MP.copy(deep = True)
    posS, MS_copy = stackRowsColumns(MS_copy, noOfRows, len(feature_names))
    posP, MP_copy = stackRowsColumns(MP_copy, noOfRows, len(feature_names))
    # MSP = pd.DataFrame(columns = feature_names)
    # for i in range(0, noOfRows):
    #     MSP.loc[posP[i]] = MS_copy.loc[posS[i]]
    return GlobalAlign(MP_copy, MS_copy, noOfRows)

generateSigns(feature_names)
df_KernelShap = pd.read_excel('signsLiverKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
df_TreeShap = pd.read_excel('signsLiverTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_LIME = pd.read_excel('signsLiverLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]

results = pd.DataFrame(columns = ['LL', 'LS', 'SL', 'SS'])
noOfInstances = df_KernelShap.shape[0]

for instance in range(0, noOfInstances):
    result = []
    data_Shap = []
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    
    data_LIME = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    
    MP = M_LIME
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_LIME
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    print(result)
    results.loc[instance] = result


results.to_excel("resultsSignsLiverMPMS_MinMax.xlsx")

[1.0, 0.9758463142762308, 0.9758463142762308, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:119: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:135: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:144: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.6970194252200251, 0.6970194252200251, 1.0]
[1.0, 0.8111473237512602, 0.8111473237512602, 1.0]
[1.0, 0.7023549609481355, 0.7023549609481355, 1.0]
[1.0, 0.8982093162017883, 0.8982093162017883, 1.0]
[1.0, 0.8968732808453559, 0.8968732808453559, 1.0]
[1.0, 0.9825243116633824, 0.9825243116633824, 1.0]
[1.0, 0.7841468417079125, 0.7841468417079125, 1.0]
[1.0, 0.979624468926867, 0.979624468926867, 1.0]
[1.0, 0.6478398994742085, 0.6478398994742085, 1.0]
[1.0, 0.8454887134214791, 0.8454887134214791, 1.0]
[1.0, 0.8638242223726854, 0.8638242223726854, 1.0]
[1.0, 0.7947219743279686, 0.7947219743279686, 1.0]
[1.0, 0.5289359644999343, 0.5289359644999343, 1.0]
[1.0, 0.924681003274174, 0.924681003274174, 1.0]
[1.0, 0.8469634481107692, 0.8469634481107692, 1.0]
[1.0, 0.8409435762490061, 0.8409435762490061, 1.0]
[1.0, 0.7211587833719716, 0.7211587833719716, 1.0]
[1.0, 0.722579971934888, 0.722579971934888, 1.0]
[1.0, 0.7478688042192648, 0.7478688042192648, 1.0]
[1.0, 0.70835347

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:119: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:144: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[nan, nan, nan, 1.0]
[1.0, 0.7685018471705368, 0.7685018471705368, 1.0]
[1.0, 0.7839690794749651, 0.7839690794749651, 1.0]
[1.0, 0.7835739477334346, 0.7835739477334346, 1.0]
[1.0, 0.6256116152757393, 0.6256116152757393, 1.0]
[1.0, 0.8703202696253701, 0.8703202696253701, 1.0]
[1.0, 0.9206283424745895, 0.9206283424745895, 1.0]
[1.0, 0.7807229125326093, 0.7807229125326093, 1.0]
[1.0, 0.9495061201411973, 0.9495061201411973, 1.0]
[1.0, 0.9886851259222466, 0.9886851259222466, 1.0]
[1.0, 0.9237577309446889, 0.9237577309446889, 1.0]
[1.0, 0.9394254051974246, 0.9394254051974246, 1.0]
[1.0, 0.8434129621957246, 0.8434129621957246, 1.0]
[1.0, 0.8968340341135763, 0.8968340341135763, 1.0]
[1.0, 0.7705391113835477, 0.7705391113835477, 1.0]
[1.0, 0.9419219818107547, 0.9419219818107547, 1.0]
[1.0, 0.9855678479600952, 0.9855678479600952, 1.0]
[1.0, 0.9292392598273969, 0.9292392598273969, 1.0]
[1.0, 0.8816108762884538, 0.8816108762884538, 1.0]
[1.0, 0.8357425146247901, 0.8357425146247901, 1.0]
[1.0, 0.93

...

[1.0, nan, nan, nan]
[1.0, 0.7511454750301519, 0.7511454750301519, 1.0]
[1.0, 0.6429302825984109, 0.6429302825984109, 1.0]
[1.0, 0.911980913548549, 0.911980913548549, 1.0]
[1.0, 0.7086372602615797, 0.7086372602615797, 1.0]
[1.0, 0.8984433934684051, 0.8984433934684051, 1.0]
[1.0, 0.7802415937569877, 0.7802415937569877, 1.0]
[1.0, 0.7300765426866712, 0.7300765426866712, 1.0]
[1.0, 0.6987473893061819, 0.6987473893061819, 1.0]
[1.0, 0.8733269962436586, 0.8733269962436586, 1.0]
[1.0, 0.9067083887278661, 0.9067083887278661, 1.0]
[1.0, 0.9545239840841957, 0.9545239840841957, 1.0]
[1.0, 0.7183827299432173, 0.7183827299432173, 1.0]
[1.0, 0.7538225124219756, 0.7538225124219756, 1.0]
[1.0, 0.604396144427476, 0.604396144427476, 1.0]
[1.0, 0.9416155936347405, 0.9416155936347405, 1.0]
[1.0, 0.7735192244899093, 0.7735192244899093, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:119: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:135: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:144: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.9864646377641241, 0.9864646377641241, 1.0]
[1.0, 0.8596893595726475, 0.8596893595726475, 1.0]
[1.0, 0.8980573308623936, 0.8980573308623936, 1.0]
[1.0, 0.7942013783637366, 0.7942013783637366, 1.0]
[1.0, 0.7937472688481206, 0.7937472688481206, 1.0]
[1.0, 0.6579852635605912, 0.6579852635605912, 1.0]
[1.0, 0.6933277524275956, 0.6933277524275956, 1.0]
[1.0, 0.8634623185337162, 0.8634623185337162, 1.0]
[1.0, 0.7843910270404638, 0.7843910270404638, 1.0]
[1.0, 0.761611618433422, 0.761611618433422, 1.0]
[1.0, 0.5214391920355582, 0.5214391920355582, 1.0]
[1.0, 0.7885046830287973, 0.7885046830287973, 1.0]
[1.0, 0.6542731759196452, 0.6542731759196452, 1.0]
[1.0, 0.6945151947520423, 0.6945151947520423, 1.0]
[1.0, 0.9714761173599394, 0.9714761173599394, 1.0]
[1.0, 0.7367877772420811, 0.7367877772420811, 1.0]
[1.0, 0.7321648340471006, 0.7321648340471006, 1.0]
[1.0, 0.9197719597402181, 0.9197719597402181, 1.0]
[1.0, 0.809153283636667, 0.809153283636667, 1.0]
[1.0, 0.954117

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:119: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_16572\3699014072.py:144: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[nan, nan, nan, 1.0]
[1.0, 0.9571594275268209, 0.9571594275268209, 1.0]
[1.0, 0.8111236370615251, 0.8111236370615251, 1.0]
[nan, nan, nan, 1.0]
